In [245]:
import sys
import os

sys.path.append(os.path.abspath("/home/miguelmd/projects/big-data-bowl-2026-analytics"))

from src.io.datasets import CSVReader
from src.core.settings import get_settings

settings = get_settings()

In [246]:
players = CSVReader().read(settings.PLAYERS_FILE)
scores = CSVReader().read(settings.SCORES_FILE)

2025-12-10 16:55:13.915 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/cleaned/players.csv
2025-12-10 16:55:13.921 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/scores.csv


In [247]:
MIN_NUM_PLAYS = 5
valid_receivers = scores.groupby("receiver_id").filter(lambda x: len(x) >= MIN_NUM_PLAYS)

In [248]:
mean_deception_by_receiver = valid_receivers.groupby("receiver_id")[
        "deception_score"
    ].mean()
sorted_mean_deception_by_receiver = mean_deception_by_receiver.sort_values(
    ascending=True
)
receivers_deception_df = sorted_mean_deception_by_receiver.reset_index().merge(
    players[["nfl_id", "player_name", "player_position"]],
    left_on="receiver_id",
    right_on="nfl_id",
    how="left",
).rename(columns={"deception_score": "Mean Deception Score", "player_name": "Name", "player_position": "Position"})
receivers_deception_df.drop(columns=["receiver_id", "nfl_id"], inplace=True)
receivers_deception_df = receivers_deception_df[["Name", "Position", "Mean Deception Score"]]

In [249]:
print("Top 10 Wide Receivers by Mean Deception Score:")
top_10_df = receivers_deception_df.head(10).copy().reset_index(drop=True)
top_10_df["Mean Deception Score"] = top_10_df["Mean Deception Score"].round(4).reset_index(drop=True)
print(top_10_df)

Top 10 Wide Receivers by Mean Deception Score:
                Name Position  Mean Deception Score
0  Wan'Dale Robinson       WR               -0.1503
1         David Bell       WR               -0.1252
2      Kalif Raymond       WR               -0.1051
3      Isaiah Likely       TE               -0.0891
4     Dalton Kincaid       TE               -0.0865
5          Noah Gray       TE               -0.0622
6     George Pickens       WR               -0.0441
7   Courtland Sutton       WR               -0.0390
8          Bo Melton       CB               -0.0378
9        Jalin Hyatt       WR               -0.0376


In [250]:
top_10_df.loc[top_10_df["Name"] == "Bo Melton", "Position"] = "WR/CB"
print(top_10_df)

                Name Position  Mean Deception Score
0  Wan'Dale Robinson       WR               -0.1503
1         David Bell       WR               -0.1252
2      Kalif Raymond       WR               -0.1051
3      Isaiah Likely       TE               -0.0891
4     Dalton Kincaid       TE               -0.0865
5          Noah Gray       TE               -0.0622
6     George Pickens       WR               -0.0441
7   Courtland Sutton       WR               -0.0390
8          Bo Melton    WR/CB               -0.0378
9        Jalin Hyatt       WR               -0.0376


In [251]:
print("Bottom 10 Wide Receivers by Mean Deception Score:")
bottom_10_df = receivers_deception_df.tail(10).copy().reset_index(drop=True)
bottom_10_df["Mean Deception Score"] = bottom_10_df["Mean Deception Score"].round(4)
print(bottom_10_df)

Bottom 10 Wide Receivers by Mean Deception Score:
                Name Position  Mean Deception Score
0       Allen Lazard       WR                0.1558
1      Brandin Cooks       WR                0.1629
2      Jake Ferguson       TE                0.1641
3      George Kittle       TE                0.1685
4        K.J. Osborn       WR                0.1779
5  Xavier Hutchinson       WR                0.1905
6         A.T. Perry       WR                0.1974
7      Curtis Samuel       WR                0.2068
8   Stephen Sullivan       TE                0.2321
9  Parker Washington       WR                0.2658


In [252]:
from plottable import ColumnDefinition, Table
from plottable.plots import image
from plottable.cmap import normed_cmap
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt

original_cmap = LinearSegmentedColormap.from_list(
    name="bugw", colors=["#ffffff", "#f2fbd2", "#c9ecb4", "#93d3ab", "#35b0ab"], N=256
)
cmap = original_cmap.reversed()

name_column = ColumnDefinition("Name", textprops={"ha": "left"}, group="Player", width=0.2)
position_column = ColumnDefinition("Position", textprops={"ha": "center"}, group="Player", width=0.2)
deception_column = ColumnDefinition("Mean Deception Score",
            cmap=normed_cmap(top_10_df["Mean Deception Score"], cmap=cmap), width=0.25)

fig, ax = plt.subplots()
plt.tight_layout(pad=0.5)
table = Table(top_10_df, column_definitions=[name_column, position_column, deception_column], index_col="Name")
fig.savefig("../assets/figures/top_10_wide_receivers.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
plt.close(fig)

In [253]:
original_cmap = LinearSegmentedColormap.from_list(
    name="red_to_white", 
    colors=["#CD5C5C", "#ffffff"],
    N=256
)
cmap = original_cmap.reversed()

name_column = ColumnDefinition("Name", textprops={"ha": "left"}, group="Player", width=0.2)
position_column = ColumnDefinition("Position", textprops={"ha": "center"}, group="Player", width=0.2)
deception_column = ColumnDefinition("Mean Deception Score",
            cmap=normed_cmap(bottom_10_df["Mean Deception Score"], cmap=cmap), width=0.25)

fig, ax = plt.subplots()
plt.tight_layout(pad=0.5)
table = Table(bottom_10_df, column_definitions=[name_column, position_column, deception_column], index_col="Name")
fig.savefig("../assets/figures/bottom_10_wide_receivers.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
plt.close(fig)